## CompanyIndexScraper Sheet
---

A sheet dedicated to the analysis and testing of scraper company indices. 

Together with an example of use.

Scraper acquires data in 6 segments:
- `COMPANY` (indexes in relation to company),
- `COMPANY_QQ` (indexes in relation to company Quarterly - Quarterly),
- `COMPANY_YY` (indexes in relation to company Year - Year),
- `SECTOR` (indexes in relation to sector),
- `SECTOR_QQ` (indexes in relation to sector Quarterly - Quarterly),
- `SECTOR_YY` (indexes in relation to sector Year - Year)

In [1]:
from scripts import CompanyIndexScraper
cis = CompanyIndexScraper()
# Scrap data - example scrap CCC (company ticker) 
cis.scrap_data("CCC")  

In [2]:
cis.get_report_index(CompanyIndexScraper.IndexType.COMPANY)

,Cena / Wartość księgowa,Cena / Wartość księgowa Grahama,Cena / Przychody ze sprzedaży,Cena / Zysk,Cena / Zysk operacyjny,EV / Przychody ze sprzedaży,EV / EBIT,EV / EBITDA,ROE,ROA,...,Wskaźnik źródeł finansowania inwestycji,Zadłużenie ogólne,Zadłużenie kapitału własnego,Zastosowanie kapitału obcego,Wskaźnik ogólnej sytuacji finansowej,I stopień pokrycia,Płynność gotówkowa,Płynność szybka,Płynność bieżąca,Pokrycie zobowiązań należnościami
2004/Q4,2.92,4.60,1.32,17.68,12.35,1.29,12.05,11.16,16.49,11.42,...,13.53,0.31,0.44,0.41,1.13,2.73,0.67,1.56,3.01,0.89
2005/Q4,7.10,10.91,3.62,27.33,21.84,3.52,21.24,19.83,25.98,21.44,...,88.54,0.18,0.21,0.25,0.70,2.86,0.94,1.69,4.16,0.75
2006/Q1,8.44,13.86,4.32,34.15,27.91,4.38,28.29,26.10,24.72,19.59,...,529.02,0.21,0.26,0.30,0.77,2.56,0.06,1.11,3.38,1.04
2006/Q2,10.16,17.21,4.44,30.02,25.34,4.49,25.65,23.89,33.84,24.58,...,104.07,0.27,0.38,0.39,0.95,2.44,0.10,0.87,2.62,0.76
2006/Q3,10.55,18.39,4.67,32.62,28.17,4.76,28.75,26.63,32.34,22.11,...,93.30,0.32,0.46,0.45,1.05,2.35,0.09,0.87,2.27,0.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023/Q1,5.05,NaN,0.26,NaN,NaN,0.65,-97.78,11.40,-71.86,-4.76,...,130.26,0.91,13.77,1.79,0.24,0.13,0.09,0.20,0.89,0.10
2023/Q2,4.48,NaN,0.38,NaN,NaN,0.74,-502.37,12.06,-34.04,-3.74,...,350.22,0.87,7.93,1.76,0.38,0.22,0.10,0.21,0.89,0.11
2023/Q3,3.42,NaN,0.29,NaN,71.38,0.65,159.56,9.67,-25.10,-2.56,...,78.74,0.88,8.66,1.66,0.36,0.22,0.12,0.22,0.85,0.10
2023/Q4,4.78,NaN,0.43,NaN,21.84,0.80,41.20,9.73,-6.67,-0.76,...,169.04,0.87,7.60,1.77,0.40,0.22,0.06,0.16,0.81,0.09


## Scrap Index Data 

In [3]:
from scripts import DatesMatcher
from scripts import CompanyIndexScraper
import pandas as pd

cis = CompanyIndexScraper()
COMPANY_TICKER_LIST = pd.read_csv("../database/designData/WIGs.csv")["WIG_ODZIEZ"].dropna().to_numpy()
D_DATE_SOURCE = pd.read_csv("../database/reportData/Quarterly_Value.csv", index_col=[0])

In [4]:

for ticker in ["CCC"]:
    d_date = D_DATE_SOURCE[D_DATE_SOURCE["Ticker"]==ticker]["Date"]
    cis.scrap_data(ticker)
    


#ticker = "CCC"
#D_DATE_SOURCE[D_DATE_SOURCE["Ticker"]==ticker]["Date"]

In [5]:
q_date = cis.get_report_index(CompanyIndexScraper.IndexType.COMPANY).index


In [11]:
for i in d_date:
    print(i)

2006-05-15
2007-05-15
2008-05-12
2009-04-30
2010-04-30
2011-04-22
2012-04-27
2013-04-30
2014-04-30
2015-04-30
2016-04-28
2017-04-28
2018-05-15
2019-05-15
2020-04-24
2021-05-18
2022-05-25
2023-06-14
2006-08-14
2007-08-10
2008-08-11
2009-08-24
2010-08-26
2011-08-29
2012-08-29
2013-08-29
2014-08-28
2015-08-25
2016-08-25
2017-08-31
2018-08-24
2019-09-03
2020-09-30
2005-10-28
2006-10-23
2007-10-31
2008-11-07
2009-11-16
2010-11-08
2011-11-10
2012-11-08
2013-11-07
2014-11-06
2015-11-06
2016-11-10
2017-11-14
2018-11-09
2019-10-30
2020-10-27
2021-10-06
2022-10-25
2023-10-11
2006-03-01
2007-03-01
2008-02-26
2009-02-19
2010-02-25
2011-02-17
2012-02-20
2013-02-20
2014-02-20
2015-02-24
2016-02-25
2017-02-28
2018-03-26
2019-03-14
2020-03-06


In [9]:
DatesMatcher.dates_matcher(q_date.to_numpy(), d_date.to_numpy())

,Q_DATES,D_DATES
0,2004/Q4,<NA>
1,2005/Q4,2005-10-28
2,2006/Q1,2006-03-01
3,2006/Q2,2006-05-15
4,2006/Q3,2006-08-14
...,...,...
70,2023/Q1,<NA>
71,2023/Q2,2023-06-14
72,2023/Q3,<NA>
73,2023/Q4,2023-10-11


678

In [38]:
import requests 

response = requests.get('https://www.notoria.pl/api/nol/news/espi-ebi?offset=0&limit=300&notoriaId=16338&source=espi&type=5&category=6') 
response.json()


{'meta': {'hasMore': False},
 'result': [{'md5': 'df961303188e76a7d8d4fc03f61b1f8e',
   'title': 'SA-QSr2 2024/2025',
   'source': 'espi',
   'ordinal': '2024',
   'published': '2024-09-24T20:20:05.000Z',
   'fileName': '20240924_222005_1652976581.xml',
   'priority': 3,
   'Issuer': {'notoriaId': 16338, 'name': 'CCC SA', 'utype': 'C'},
   'types': [{'id': 5, 'name': 'finanse'}]},
  {'md5': 'ead69ab1e12cef7825d9d13693392c4d',
   'title': 'SA-QSr1 2024/2025',
   'source': 'espi',
   'ordinal': '1/2024',
   'published': '2024-06-12T15:12:37.000Z',
   'fileName': '20240612_171237_2015672715.xml',
   'priority': 3,
   'Issuer': {'notoriaId': 16338, 'name': 'CCC SA', 'utype': 'C'},
   'types': [{'id': 5, 'name': 'finanse'}]},
  {'md5': '30cb203fb5874bc6304f671175783c0a',
   'title': 'SA-R 2023/2024',
   'source': 'espi',
   'ordinal': '2023',
   'published': '2024-04-05T20:32:42.000Z',
   'fileName': '20240405_223242_1344896505.xml',
   'priority': 3,
   'Issuer': {'notoriaId': 16338, 'name

In [164]:
ticker = "ATR"

In [165]:
import difflib


In [150]:
import numpy as np
import requests
import re


class DateScraper:
    @staticmethod
    def __cast_espi_title(title):
        year = re.findall("\\d{4}", title)

        if "SA-R" in title or "SA-RS" in title:
            return f"{year[0]}/R"
        elif "QSr1" in title or "QS 1" in title or "Q1" in title or "Q 1" in title:
            return f"{year[0]}/Q1"
        elif "QSr2" in title or "QS 2" in title or "Q2" in title or "Q 2" in title:
            return f"{year[0]}/Q2"
        elif "QSr3" in title or "QS 3" in title or "Q3" in title or "Q 3" in title:
            return f"{year[0]}/Q3"
        elif "QSr4" in title or "QS 4" in title or "Q4" in title or "Q 4" in title:
            return f"{year[0]}/Q4"
        
    @staticmethod
    def __check_request_status_code(response):
        try:
            response.raise_for_status()
        except requests.exceptions.HTTPError as e:
            return "Error: " + str(e)
        
    @staticmethod
    def get_date_map(ticker):
        # GET company id
        response = requests.get(f"https://www.notoria.pl/api/nol/search/news-issuer?phrase={ticker}") 
        DateScraper.__check_request_status_code(response)
        notoriaId = response.json()[0]["id"]

        # GET financial ESPI data 
        response = requests.get(f"https://www.notoria.pl/api/nol/news/espi-ebi?offset=0&limit=1000&notoriaId={notoriaId}&source=espi&type=5&category=6")
        DateScraper.__check_request_status_code(response)

        # Data customization
        espi_reports = pd.DataFrame(response.json()["result"][::-1])[["title", "published"]]
        espi_reports["published"] = espi_reports["published"].apply(lambda x : np.datetime64(x[:10]))
        espi_reports["title"] = espi_reports["title"].apply(lambda x : DateScraper.__cast_espi_title(x))
        espi_reports = espi_reports.dropna()
        espi_reports = espi_reports.drop_duplicates(subset="title")
        espi_reports.columns = ["Q_DATES", "D_DATES"]


200